# **Ayudantía 7: Networking**


Autora: Camila González P. ([@camilagonzalezp](https://github.com/camilagonzalezp))

## Sockets

### ¿Qué es?

Es un objeto del sistema operativo que permite a un programa transmitir y recibir datos desde y hacia otro programa.

### ¿Cómo se usa?

#### Cliente TCP

Para crear un socket, se debe crear una instancia de la clase `socket(family, type)`.
* `family`: tipo de dirección IP a usar.
    * Para direcciones IPv4: `AF_INET`.
    * Para direcciones IPv6: `AF_INET6`.
* `type`: protocolo de transporte a usar.
    * Para protocolo TCP: `SOCK_STREAM`.
    * Para protocolo UDP: `SOCK_DGRAM`.

Para conectarse a un servidor TCP se usa el método `connect((host, port))` del socket. 
* `host`: dirección IP del host destinatario.
* `port`: puerto del host destinatario.

Para enviar un mensaje, se usa el método `sendall(bytes)` de socket.

Para recibir un mensaje, se usa el método `recv(buffer)->bytes` de socket.
* `buffer`: cantidad máxima de bytes a recibir.

Para cerrar la conexión se usa el método `close()` de socket.

In [ ]:
import socket

socket_cliente = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

HOST = socket_cliente.gethostname() 
# permite obtener la dirección IP del host donde se está ejecutando el programa.
# HOST = "localhost"
PORT = 5000

try:
    socket_cliente.connect((HOST, PORT))
    mensaje_a_enviar = 'Mensaje enviado desde cliente a servidor'
    socket_cliente.sendall(mensaje_a_enviar.encode('utf-8'))
    bytes_recibidos = socket_cliente.recv(4096)
    print(f"Mensaje recibido: {bytes_recibidos.decode('utf-8')}")
except ConnectionError as error:
    print('La conexión ha fallado :o')
    print(f'Error: {error}')
finally:  
    socket_cliente.close()

#### Servidor TCP

Se crea un socket de la misma forma que en el cliente, con `socket(family, type)`.

Para enlazar el socket a un puerto se usa el método `bind((host, port))`.
* `host`:  host en el que se está corriendo el servidor.
* `port`: puerto donde se va a escuchar las conexiones.

Para escuchar conexiones se usa el método `listen()`.

Para aceptar conexiones se usa el método `accept()->socket_cliente, address`.
* `socket_cliente`: socket que permite al servidor comunicarse con el cliente que se acepta.
* `address`: tupla con la dirección IP y el puerto del cliente.

Para cerrar el servidor se usa el método `close()` de socket.

In [ ]:
import socket

socket_servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

HOST = socket_servidor.gethostname()
PORT = 5000

socket_servidor.bind((HOST, PORT))
socket_servidor.listen()

while True:
    try:
        socket_cliente, adress = socket_servidor.accept()
        print(f'Conexión aceptada desde: {adress}')
        bytes_recibidos = socket_cliente.recv(4096)
        print(f"Mensaje recibido: {bytes_recibidos.decode('utf-8')}")
        mensaje_a_enviar = 'Mensaje enviado desde servidor a cliente'
        socket_cliente.sendall(mensaje_a_enviar.encode('utf-8'))
    except ConnectionError as error:
        print('La conexión ha fallado :o')
        print(f'Error: {error}')
    finally:
        socket_servidor.close()

## Envío de información: protocolos

### ¿Qué es?

Es un esquema de comunicación para la transferencia de información.

### ¿Para qué sirve?

* Envío de grandes cantidades de datos.
* Estandarización del envío de información.
* Orden en la transferencia de datos.
* Garantiza el correcto traspaso de información.

### ¿Qué información puede contener un protocolo?

* Tamaño de los datos enviados.
* Tamaño de los chunks.
* Orden de los bytes.
* Alfabeto de codificación.

### Ejemplo!

In [1]:
import json

TAMANO_CHUNK = 4

mensaje = {
    'nombre': 'Margot',
    'edad': 15,
    'raza': 'Schnauzer'
}

bytes_mensaje = json.dumps(mensaje).encode('utf-8')

while len(bytes_mensaje) % TAMANO_CHUNK != 0:
    bytes_mensaje += b'\x00'

numero_chunks = len(bytes_mensaje) // TAMANO_CHUNK
bytes_numero_chunks = numero_chunks.to_bytes(4, 'little')

mensaje_final = bytes_numero_chunks + bytes_mensaje
print(f'Mensaje final: {mensaje_final}')

Mensaje final: b'\x0e\x00\x00\x00{"nombre": "Margot", "edad": 15, "raza": "Schnauzer"}\x00\x00\x00'


In [2]:
numero_chunks = int.from_bytes(mensaje_final[:4], 'little')

chunks_recibidos = 0
bytes_recibidos = b''
actual = 4

while chunks_recibidos < numero_chunks:
    chunk = mensaje_final[actual:actual+TAMANO_CHUNK]
    bytes_recibidos += chunk
    actual += TAMANO_CHUNK
    chunks_recibidos += 1
print(f'Bytes recibidos: {bytes_recibidos}')

bytes_recibidos_limpios = bytes_recibidos.strip(b"\x00")
mensaje_recibido = json.loads(bytes_recibidos_limpios)
print(f'Mensaje recibido: {mensaje_recibido}')

Bytes recibidos: b'{"nombre": "Margot", "edad": 15, "raza": "Schnauzer"}\x00\x00\x00'
Mensaje recibido: {'nombre': 'Margot', 'edad': 15, 'raza': 'Schnauzer'}


## Manejo de múltiples clientes en forma concurrente 😱

* El manejo de varios usuarios simultáneamente se logra mediante el uso de **threads**.
* Un thread encargado de aceptar nuevos clientes y un thread encargado de cada cliente.

### Ejemplo final 🎉

#### Servidor

In [ ]:
class Servidor:

    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.socket_servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket_servidor.bind((self.host, self.port))
        self.socket_servidor.listen()
        print(f'Servidor escuchando en {self.host}:{self.port}')

        thread = threading.Thread(target=self.aceptar_clientes)
        thread.start()

In [ ]:
    def aceptar_clientes(self):
        while True:
            socket_cliente, address = self.socket_servidor.accept()
            print(f'Cliente con dirección {address} se ha conectado al servidor')
            thread_cliente = threading.Thread(target=self.escuchar_cliente, args=(socket_cliente,), daemon=True)
            thread_cliente.start()

In [ ]:
    def escuchar_cliente(self, socket_cliente):
        try:
            while True:
                mensaje = self.recibir(socket_cliente)
                respuesta = self.manejar_mensaje_recibido(mensaje)
                print(f'Enviando respuesta: {respuesta}')
                self.enviar(respuesta, socket_cliente)
        except ConnectionResetError:
            print('Error de conexión con el cliente')

In [ ]:
    def recibir(self, socket_cliente):
        largo_bytes_mensaje = socket_cliente.recv(4)
        largo_mensaje = int.from_bytes(largo_bytes_mensaje, byteorder='big')
        bytes_mensaje = bytearray()
        while len(bytes_mensaje) < largo_mensaje:
            bytes_mensaje += socket_cliente.recv(60)
        bytes_mensaje_limpios = bytes_mensaje.strip(b'\x00')
        mensaje = self.decodificar_mensaje(bytes_mensaje_limpios)
        return mensaje

In [ ]:
    def enviar(self, mensaje, socket_cliente):
        bytes_mensaje = self.codificar_mensaje(mensaje)
        while len(bytes_mensaje) % 60 != 0:
            bytes_mensaje += b'\x00'
        largo_bytes_mensaje = len(bytes_mensaje).to_bytes(4, byteorder='big')
        socket_cliente.sendall(largo_bytes_mensaje + bytes_mensaje)

In [ ]:
    def codificar_mensaje(self, mensaje):
        try:
            mensaje_json = json.dumps(mensaje)
            bytes_mensaje = mensaje_json.encode('utf-8')
            return bytes_mensaje
        except json.JSONDecodeError:
            print('No se pudo codificar el mensaje')
            return b''

In [ ]:
    def decodificar_mensaje(self, bytes_mensaje):
        try:
            mensaje = json.loads(bytes_mensaje)
            return mensaje
        except json.JSONDecodeError:
            print('No se pudo decodificar el mensaje')
            return ''

In [ ]:
    def manejar_mensaje_recibido(self, mensaje):
        return f'Respuesta asociada al mensaje {mensaje}'

#### Cliente

In [ ]:
class Cliente:

    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.socket_cliente = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            self.socket_cliente.connect((self.host, self.port))
            thread = threading.Thread(target=self.escuchar_servidor, daemon=True)
            thread.start()
            self.recibir_input()
        except ConnectionRefusedError:
            self.socket_cliente.close()

In [ ]:
    def recibir_input(self):
        while True:
            mensaje = input()
            print(f'Enviando mensaje: {mensaje}')
            self.enviar(mensaje)

In [ ]:
    def escuchar_servidor(self):
        try:
            while True:
                mensaje = self.recibir()
                print(f'Mensaje recibido: {mensaje}')
                self.manejar_mensaje_recibido(mensaje)
        except ConnectionResetError:
            print("Error de conexión con el servidor")
        finally:
            self.socket_cliente.close()

In [ ]:
    def recibir(self):
        largo_bytes_mensaje = self.socket_cliente.recv(4)
        largo_mensaje = int.from_bytes(largo_bytes_mensaje, byteorder='big')
        bytes_mensaje = bytearray()
        while len(bytes_mensaje) < largo_mensaje:
            bytes_mensaje += self.socket_cliente.recv(60)
        bytes_mensaje_limpios = bytes_mensaje.strip(b'\x00')
        mensaje = self.decodificar_mensaje(bytes_mensaje_limpios)
        return mensaje

In [ ]:
    def enviar(self, mensaje):
        bytes_mensaje = self.codificar_mensaje(mensaje)
        while len(bytes_mensaje) % 60 != 0:
            bytes_mensaje += b'\x00'
        largo_bytes_mensaje = len(bytes_mensaje).to_bytes(4, byteorder='big')
        self.socket_cliente.sendall(largo_bytes_mensaje + bytes_mensaje)

In [ ]:
    def codificar_mensaje(self, mensaje):
        try:
            mensaje_json = json.dumps(mensaje)
            bytes_mensaje = mensaje_json.encode('utf-8')
            return bytes_mensaje
        except json.JSONDecodeError:
            print('No se pudo codificar el mensaje')
            return b''

In [ ]:
    def decodificar_mensaje(self, bytes_mensaje):
        try:
            mensaje = json.loads(bytes_mensaje)
            return mensaje
        except json.JSONDecodeError:
            print('No se pudo decodificar el mensaje')
            return ''

In [ ]:
    def manejar_mensaje_recibido(self, mensaje):
        print(f'Acción asociada al mensaje: {mensaje}') 